In [3]:
import numpy as np
import matplotlib.pyplot as plt
#####################################################################################################################
def sigmoid(x):
    """ Sigmoid function.
    This function accepts any shape of np.ndarray object as input and perform sigmoid operation.
    """
    return 1 / (1 + np.exp(-x))


def der_sigmoid(y):
    """ First derivative of Sigmoid function.
    The input to this function should be the value that output from sigmoid function.
    """
    return sigmoid(y) * (1 - sigmoid(y))
#####################################################################################################################
class GenData:
    @staticmethod
    def _gen_linear(n=100):
        """ Data generation (Linear)

        Args:
            n (int):    the number of data points generated in total.

        Returns:
            data (np.ndarray, np.float):    the generated data with shape (n, 2). Each row represents
                a data point in 2d space.
            labels (np.ndarray, np.int):    the labels that correspond to the data with shape (n, 1).
                Each row represents a corresponding label (0 or 1).
        """
        data = np.random.uniform(0, 1, (n, 2))

        inputs = []
        labels = []

        for point in data:
            inputs.append([point[0], point[1]])

            if point[0] > point[1]:
                labels.append(0)
            else:
                labels.append(1)

        return np.array(inputs), np.array(labels).reshape((-1, 1))

    @staticmethod
    def _gen_xor(n=100):
        """ Data generation (XOR)

        Args:
            n (int):    the number of data points generated in total.

        Returns:
            data (np.ndarray, np.float):    the generated data with shape (n, 2). Each row represents
                a data point in 2d space.
            labels (np.ndarray, np.int):    the labels that correspond to the data with shape (n, 1).
                Each row represents a corresponding label (0 or 1).
        """
        data_x = np.linspace(0, 1, n // 2)

        inputs = []
        labels = []

        for x in data_x:
            inputs.append([x, x])
            labels.append(0)

            if x == 1 - x:
                continue

            inputs.append([x, 1 - x])
            labels.append(1)

        return np.array(inputs), np.array(labels).reshape((-1, 1))

    @staticmethod
    def fetch_data(mode, n):
        """ Data gather interface

        Args:
            mode (str): 'Linear' or 'XOR', indicate which generator is used.
            n (int):    the number of data points generated in total.
        """
        assert mode == 'Linear' or mode == 'XOR'

        data_gen_func = {
            'Linear': GenData._gen_linear,
            'XOR': GenData._gen_xor
        }[mode]

        return data_gen_func(n)
#####################################################################################################################
class SimpleNet:
    def __init__(self, num_step=2000, print_interval=100):
        """ A hand-crafted implementation of simple network.

        Args:
            num_step (optional):    the total number of training steps.
            print_interval (optional):  the number of steps between each reported number.
        """
        self.num_step = num_step
        self.print_interval = print_interval

        # Model parameters initialization
        # hidden layer 1: 100 nodes
        # hidden layer 2: 10 nodes
        # Please initiate your network parameters here.
        
        # 輸入node數
        InputNodes = 2
        # hidden layer1的node數
        HiddenLayerNodes1 = 100
        # hidden layer1的node數
        HiddenLayerNodes2 = 10
        # 輸出node數
        OutputNodes = 1
        
        self.hidden1_weights = np.random.normal(size = (InputNodes, HiddenLayerNodes1)) #w1 #2*100
        self.hidden2_weights = np.random.normal(size = (HiddenLayerNodes1, HiddenLayerNodes2)) #w2 #100*10
        self.output_weights = np.random.normal(size = (HiddenLayerNodes2, OutputNodes)) #w3 #10*1     

    @staticmethod
    def plot_result(self, data, gt_y, pred_y):
        """ Data visualization with ground truth and predicted data comparison. There are two plots
        for them and each of them use different colors to differentiate the data with different labels.

        Args:
            data:   the input data
            gt_y:   ground truth to the data
            pred_y: predicted results to the data
        """
        assert data.shape[0] == gt_y.shape[0]
        assert data.shape[0] == pred_y.shape[0]

        plt.figure()
        
        plt.subplot(1, 2, 1)
        plt.title('Ground Truth', fontsize=18)

        for idx in range(data.shape[0]):
            if gt_y[idx] == 0:
                plt.plot(data[idx][0], data[idx][1], 'ro')
            else:
                plt.plot(data[idx][0], data[idx][1], 'bo')

        plt.subplot(1, 2, 2)
        plt.title('Prediction', fontsize=18)

        for idx in range(data.shape[0]):
            if pred_y[idx] == 0:
                plt.plot(data[idx][0], data[idx][1], 'ro')
            else:
                plt.plot(data[idx][0], data[idx][1], 'bo')
        plt.show()
        plt.close()
        
        plt.figure()

        plt.title('Training Loss Plots', fontsize=18)

        epoch = self.epoch
        plt.plot(epoch, self.loss)
        
        plt.show()

    def forward(self, inputs):
        """ Implementation of the forward pass.
        It should accepts the inputs and passing them through the network and return results.
        """

        #hidden layer 1 - X to Z1
        self.Z1 = np.dot(inputs, self.hidden1_weights) #A1 #1*100
        self.Z1_sig = sigmoid(self.Z1) #Z1 

        #hidden layer 2 - Z1 to Z2
        self.Z2 = np.dot(self.Z1_sig, self.hidden2_weights) #A2 #1*10
        self.Z2_sig = sigmoid(self.Z2) #Z2
        
        #output - Z2 to Y
        self.Y = np.dot(self.Z2_sig, self.output_weights) #A3 #1*1
        self.Y_sig = sigmoid(self.Y) #Z3

        return self.Y_sig

    def backward(self, inputs):
        """ Implementation of the backward pass.
        It should utilize the saved loss to compute gradients and update the network all the way to the front.
        """
        learning_rate = 1e-2

        self.Z3_derSig = der_sigmoid(self.Y) #1*1
        self.Z2_derSig = der_sigmoid(self.Z2) #1*10
        self.Z1_derSig = der_sigmoid(self.Z1) #1*100
        
        #Y to Z2
        self.YtoZ2_1 = np.dot(self.error, self.Z3_derSig.T) #1*der_sig(A3) #(1*1)(1*1)=(1*1)
        self.YtoZ2 = np.dot(self.YtoZ2_1, self.Z2_sig) #1*der_sig(A3)*Z2 #(1*1)(1*10) = (1*10)
        #Z2 to Z1
        self.Z2toZ1_1 = np.dot(self.output_weights, self.YtoZ2_1) #1*der_sig(A3)*w3 #(1*1)(1*10)=(1*10) #數組與矩陣相乘，輸出矩陣大小        
        self.Z2toZ1_2 = np.multiply(self.Z2toZ1_1, self.Z2_derSig.T) #1*der_sig(A3)*w3*der_sig(A2) #(10*1)(1*10)=(1*10)
        self.Z2toZ1 = np.dot(self.Z2toZ1_2, self.Z1_sig) #1*der_sig(A3)*w3*der_sig(A2) * Z1 #(1*10)(1*100)=(1*100)
        
        #Z1 to X
        self.Z1toX_1 = np.dot(self.hidden2_weights, self.Z2toZ1_2) #1*der_sig(A3)*w3*der_sig(A2) * w2 #(1*10)(10*100)= 1*100
        self.Z1toX_2 = np.multiply(self.Z1toX_1, self.Z1_derSig.T) #1*der_sig(A3)*w3*der_sig(A2) * w2 * der_sig(A1) #(100*10)(1*100)=(1*100)
        self.Z1toX = np.dot(self.Z1toX_2, inputs) #1*der_sig(A3)*w3*der_sig(A2) * w2 * der_sig(A1) * x #(10*100)(1*2)

        self.hidden1_weights = self.hidden1_weights - learning_rate * self.Z1toX.T
        self.hidden2_weights = self.hidden2_weights - learning_rate * self.Z2toZ1.T
        self.output_weights = self.output_weights - learning_rate * self.YtoZ2.T

    def train(self, inputs, labels):
        """ The training routine that runs and update the model.

        Args:
            inputs: the training (and testing) data used in the model.
            labels: the ground truth of correspond to input data.
        """
        # make sure that the amount of data and label is match
        assert inputs.shape[0] == labels.shape[0]

        n = inputs.shape[0]
        
        self.loss = []
        self.epoch = []
        
        for epochs in range(self.num_step):
            for idx in range(n):
                # operation in each training step:
                #   1. forward passing
                #   2. compute loss
                #   3. propagate gradient backward to the front
                self.output = self.forward(inputs[idx:idx+1, :])
                self.error = self.output - labels[idx:idx+1, :]
                self.backward(inputs[idx:idx+1, :])

            if epochs % self.print_interval == 0:
                
                if(epochs % 3 == 0):
                    print()
                
                print('Epochs {}: '.format(epochs), end=' ')
                self.test(inputs, labels)
                
                self.epoch.append(epochs)
                self.loss.append(abs(self.error[0]))
        
        print('Training finished')
        self.test(inputs, labels)

    def test(self, inputs, labels):
        """ The testing routine that run forward pass and report the accuracy.

        Args:
            inputs: the testing data. One or several data samples are both okay.
                The shape is expected to be [BatchSize, 2].
            labels: the ground truth correspond to the inputs.
        """
        n = inputs.shape[0]

        error = 0.0
        for idx in range(n):
            result = self.forward(inputs[idx:idx+1, :])
            error += abs(result - labels[idx:idx+1, :])

        error /= n

        """ Print or plot your results in your preferred forms"""

        print('accuracy: %.2f' % ((1 - error)*100) + '%',end=', ')
        print('loss: %.6f' % error,end='| ')

# Run "Linear"

In [ ]:
if __name__ == '__main__':
    """ Customize your own code if needed """

    data, label = GenData.fetch_data('Linear', 100)
    net = SimpleNet(500000, 1000)
    net.train(data, label)

    pred_result = np.round(net.forward(data))
    SimpleNet.plot_result(net, data, label, pred_result)


Epochs 0:  accuracy: 49.61%, loss: 0.503857| Epochs 1000:  accuracy: 93.94%, loss: 0.060556| Epochs 2000:  accuracy: 96.33%, loss: 0.036691| 
Epochs 3000:  accuracy: 97.32%, loss: 0.026831| Epochs 4000:  accuracy: 97.87%, loss: 0.021315| Epochs 5000:  accuracy: 98.22%, loss: 0.017752| 
Epochs 6000:  accuracy: 98.47%, loss: 0.015250| Epochs 7000:  accuracy: 98.66%, loss: 0.013394| Epochs 8000:  accuracy: 98.80%, loss: 0.011962| 
Epochs 9000:  accuracy: 98.92%, loss: 0.010825| Epochs 10000:  accuracy: 99.01%, loss: 0.009899| Epochs 11000:  accuracy: 99.09%, loss: 0.009131| 
Epochs 12000:  accuracy: 99.15%, loss: 0.008483| Epochs 13000:  accuracy: 99.21%, loss: 0.007930| Epochs 14000:  accuracy: 99.25%, loss: 0.007452| 
Epochs 15000:  accuracy: 99.30%, loss: 0.007035| Epochs 16000:  accuracy: 99.33%, loss: 0.006667| Epochs 17000:  accuracy: 99.37%, loss: 0.006341| 
Epochs 18000:  accuracy: 99.40%, loss: 0.006049| Epochs 19000:  accuracy: 99.42%, loss: 0.005786| Epochs 20000:  accuracy: 9

# run "XOR"

In [ ]:
if __name__ == '__main__':
    """ Customize your own code if needed """
    
    data, label = GenData.fetch_data('XOR', 100)

    net = SimpleNet(1000000, 1000)
    net.train(data, label)

    pred_result = np.round(net.forward(data))
    SimpleNet.plot_result(net, data, label, pred_result)